In [79]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from lib.residental_data_source import ResidentialDataSource
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [63]:
data_source = ResidentialDataSource('src/dataset_prepared.xlsx')
ds = data_source.fetch_data_set()

In [64]:
def standardize_numeric_data(ds, target_column='Таргет'):

    numerical_data = ds.select_dtypes(include=['int64', 'float64']).drop(columns=[target_column])

    std = StandardScaler()
    std_data = std.fit_transform(numerical_data)
    cols = std.get_feature_names_out()
    
    std_ds = pd.DataFrame(std_data, columns=cols)
    ds = pd.concat([ds.drop(columns=cols), std_ds], axis=1)
    
    return ds


In [66]:
ds = standardize_numeric_data(ds, target_column='Таргет')

In [67]:
cols = ['Есть кладбище 250м', 'Есть наркодиспансер 250м']
ds.drop(columns=cols, inplace=True)


In [68]:
object_list = [
    'Инфраструктура: Рядом ВУЗ', 
    'Есть железная дорога 250м',
    'Есть ЛЭП 100м',
    'Есть магистральные дороги 100м',
    'Есть АЗС 100м',
    'Есть скверы 500м',
    'Есть лес 500м',
    'Есть водоем 500м',
    'Есть пляж 1000м',
    'Есть тюрьма 250м',
    'Есть ПНД 250м',
    'Есть промзона 250м',
    'Есть гаражи 500м',
    'Уехать без пересадок: Луговая',
    'Уехать без пересадок: Центр',
    'Уехать без пересадок: Первая речка',
    'Уехать без пересадок: Вторая речка',
    'Уехать без пересадок: Третья рабочая',
    'Уехать без пересадок: Окатовая',
    'Уехать без пересадок: Баляева',
    'Уехать без пересадок: Кампус ДВФУ',
    'Внутренний двор без машин', 
    'Видеонаблюдение во дворе', 
    'Закрытая территория', 
    'Инфраструктура: Рядом ВУЗ', 
    'Всероссийский застройщик', 
    'Станции электропоезда 1000м']

In [69]:
ids = (ds.dtypes == object)
str_cols = ds.columns[ids]

for col in str_cols:
    ds[col] = ds[col].str.lower().str.strip()

In [71]:
for col in object_list:
    num_unique = len(ds[col].unique())
    if num_unique != 2:
        print(f"Column '{col}' has {num_unique} unique values: {ds[col].unique()}")


Column 'Внутренний двор без машин' has 3 unique values: ['да' 'нет' 'неизвестно']
Column 'Видеонаблюдение во дворе' has 3 unique values: ['неизвестно' 'да' 'нет']
Column 'Закрытая территория' has 3 unique values: ['да' 'нет' 'неизвестно']


In [75]:
def preprocess_categorical_columns(ds, object_columns):
    for col in object_columns:
        if ds[col].dtype == 'object' and 'неизвестно' in ds[col].unique():
            target_yes = ds.loc[ds[col] == 'да', 'Таргет']
            target_no = ds.loc[ds[col] == 'нет', 'Таргет']
            median_yes = target_yes.median()
            median_no = target_no.median()
            unknown_indices = ds[ds[col] == 'неизвестно'].index

            for ind in unknown_indices:
                target_value = ds.loc[ind, 'Таргет']
                dist_yes = abs(target_value - median_yes)
                dist_no = abs(target_value - median_no)
                
                if dist_yes < dist_no:
                    ds.loc[ind, col] = 'да'
                else:
                    ds.loc[ind, col] = 'нет'
                    
    return ds


preprocess_categorical_columns(ds, object_list)

,Населенный пункт,Группа компаний,Класс недвижимости,Статус,Месяц,Таргет,ЖК,Дата получения РС,Дата публикации ИНС,Всероссийский застройщик,...,Расстояние по навигатору до: Артем,Расстояние по навигатору до: В-надеждинское,Вид из окна на море,"Общая сумма полученных денег с проданных квартир, руб",Количество проданных машиномест,Количество проданных квартир,Общая площадь проданных квартир,Процент распроданности,Кумулятивная средняя стоимость кв.метра,Количество проданных нежилых помещений
0,артем,сзвосход,типовой,строится,апрель 2021,0.000000,восход 40376,2020-08-19,2021-04-16,нет,...,-3.081237,-1.643038,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,артем,сзвосход,типовой,строится,май 2021,0.000000,восход 40376,2020-08-19,2021-04-16,нет,...,-3.081237,-1.643038,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,артем,сзвосход,типовой,строится,июнь 2021,0.000000,восход 40376,2020-08-19,2021-04-16,нет,...,-3.081237,-1.643038,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,артем,сзвосход,типовой,строится,июль 2021,0.000000,восход 40376,2020-08-19,2021-04-16,нет,...,-3.081237,-1.643038,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,артем,сзвосход,типовой,строится,август 2021,95000.000000,восход 40376,2020-08-19,2021-04-16,нет,...,-3.081237,-1.643038,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1602,вольно-надеждинское,сз днс дом,комфорт,строится,апрель 2022,0.000000,формат 44614,2021-06-22,2021-07-05,нет,...,-1.465829,-2.789875,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1603,вольно-надеждинское,сз днс дом,комфорт,строится,май 2022,0.000000,формат 44614,2021-06-22,2021-07-05,нет,...,-1.465829,-2.789875,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1604,вольно-надеждинское,сз днс дом,комфорт,строится,июнь 2022,0.000000,формат 44614,2021-06-22,2021-07-05,нет,...,-1.465829,-2.789875,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1605,вольно-надеждинское,сз днс дом,комфорт,строится,июль 2022,0.000000,формат 44614,2021-06-22,2021-07-05,нет,...,-1.465829,-2.789875,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
for col in object_list:
    num_unique = len(ds[col].unique())
    if num_unique != 2:
        print(f"Column '{col}' has {num_unique} unique values: {ds[col].unique()}")

In [77]:
def standardize_categorical_data(ds, object_columns):
    
    for col in object_columns:
        ds.loc[ds[col] == 'нет', col] = 0
        ds.loc[ds[col] == 'да', col] = 1
        ds[col] = ds[col].astype(int)
    
    return ds


In [78]:
standardize_categorical_data(ds, object_list)

,Населенный пункт,Группа компаний,Класс недвижимости,Статус,Месяц,Таргет,ЖК,Дата получения РС,Дата публикации ИНС,Всероссийский застройщик,...,Расстояние по навигатору до: Артем,Расстояние по навигатору до: В-надеждинское,Вид из окна на море,"Общая сумма полученных денег с проданных квартир, руб",Количество проданных машиномест,Количество проданных квартир,Общая площадь проданных квартир,Процент распроданности,Кумулятивная средняя стоимость кв.метра,Количество проданных нежилых помещений
0,артем,сзвосход,типовой,строится,апрель 2021,0.000000,восход 40376,2020-08-19,2021-04-16,0,...,-3.081237,-1.643038,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,артем,сзвосход,типовой,строится,май 2021,0.000000,восход 40376,2020-08-19,2021-04-16,0,...,-3.081237,-1.643038,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,артем,сзвосход,типовой,строится,июнь 2021,0.000000,восход 40376,2020-08-19,2021-04-16,0,...,-3.081237,-1.643038,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,артем,сзвосход,типовой,строится,июль 2021,0.000000,восход 40376,2020-08-19,2021-04-16,0,...,-3.081237,-1.643038,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,артем,сзвосход,типовой,строится,август 2021,95000.000000,восход 40376,2020-08-19,2021-04-16,0,...,-3.081237,-1.643038,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1602,вольно-надеждинское,сз днс дом,комфорт,строится,апрель 2022,0.000000,формат 44614,2021-06-22,2021-07-05,0,...,-1.465829,-2.789875,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1603,вольно-надеждинское,сз днс дом,комфорт,строится,май 2022,0.000000,формат 44614,2021-06-22,2021-07-05,0,...,-1.465829,-2.789875,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1604,вольно-надеждинское,сз днс дом,комфорт,строится,июнь 2022,0.000000,формат 44614,2021-06-22,2021-07-05,0,...,-1.465829,-2.789875,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1605,вольно-надеждинское,сз днс дом,комфорт,строится,июль 2022,0.000000,формат 44614,2021-06-22,2021-07-05,0,...,-1.465829,-2.789875,-0.457659,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
object_ds = ds.select_dtypes(include=['object'])
object_columns = object_ds.columns

multi_categorical_cols = list(set(object_ds.columns.tolist()) - set(object_list))
object_ds[multi_categorical_cols].head()


,Вспомогательный материал фасада,Район,Есть квартиры с черновой отделкой,Есть площадки для собак,ЖК,Группа компаний,Есть велопарковка,Статус,Месяц,Тип дома,Есть квартиры под ключ,Есть квартиры с чистовой отделкой,Тип фасада,Класс недвижимости,Основной материал фасада,Есть квартиры без отделки,Населенный пункт
0,стекло,артем,да,нет,восход 40376,сзвосход,да,строится,апрель 2021,монолит,да,да,панель,типовой,камень,да,артем
1,стекло,артем,да,нет,восход 40376,сзвосход,да,строится,май 2021,монолит,да,да,панель,типовой,камень,да,артем
2,стекло,артем,да,нет,восход 40376,сзвосход,да,строится,июнь 2021,монолит,да,да,панель,типовой,камень,да,артем
3,стекло,артем,да,нет,восход 40376,сзвосход,да,строится,июль 2021,монолит,да,да,панель,типовой,камень,да,артем
4,стекло,артем,да,нет,восход 40376,сзвосход,да,строится,август 2021,монолит,да,да,панель,типовой,камень,да,артем


In [82]:
def encode_categorical_features(ds, cols):

    ohe = OneHotEncoder()
    ohe.fit(ds[cols])
    ohe_data = ohe.transform(ds[cols]).toarray()
    ohe_cols = ohe.get_feature_names_out()
    ohe_df = pd.DataFrame(ohe_data, columns=ohe_cols)

    ds = pd.concat([ds.drop(cols, axis=1), ohe_df], axis=1)
    return ds

encode_categorical_features(object_ds, multi_categorical_cols)

,Вспомогательный материал фасада_дерево,Вспомогательный материал фасада_другой,Вспомогательный материал фасада_керамогранит,Вспомогательный материал фасада_кирпич,Вспомогательный материал фасада_металл,Вспомогательный материал фасада_нет,Вспомогательный материал фасада_пластик,Вспомогательный материал фасада_стекло,Район_cнеговая падь,Район_артем,...,Основной материал фасада_кирпич,Основной материал фасада_металл,Основной материал фасада_пластик,Основной материал фасада_стекло,Есть квартиры без отделки_да,Есть квартиры без отделки_неизвестно,Есть квартиры без отделки_нет,Населенный пункт_артем,Населенный пункт_владивосток,Населенный пункт_вольно-надеждинское
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1602,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1603,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1604,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1605,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [83]:
ds.to_excel('src/dataset_processed.xlsx', index=False)